In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding
from keras.preprocessing.sequence import pad_sequences
filepath=[r"B:\project\human\human_chunk_12.csv",r"B:\project\ai\ai_chunk_12.csv"]
dfs = [pd.read_csv(filepath) for filepath in filepath]
df = pd.concat(dfs, ignore_index=True)
print(df.shape)


(28538, 2)


In [7]:
df

,text,generated
0,Google has had Ears that EAN drive independent...,0.0
1,It has been shown in scientific studies that p...,0.0
2,Car have become a part of everyday life over t...,0.0
3,Humans are slowly destroying earth and scienti...,0.0
4,Dear Principal I really thank that students sh...,0.0
...,...,...
28533,I agree that online or video Conferencing dist...,1.0
28534,The Benefits of Limiting Car Usage Many cities...,1.0
28535,As an eighth grade student I believe that atte...,1.0
28536,It is a common belief that having a broad know...,1.0


In [2]:
essay=df['text'].tolist()
from transformers import LongformerTokenizer
tokenizer=LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
essay=df['text'].tolist()
tokens=tokenizer(essay,return_tensors='pt',padding=True,truncation=True)

In [3]:
x=tokens['input_ids']
y=df['generated']
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=42)
# input_ids_size = tokens['input_ids'].size()
# attention_mask_size = tokens['attention_mask'].size()
# print("Input IDs size:", input_ids_size)
# print("Attention Mask size:", attention_mask_size)
# # Assuming 'tokens' is the result of tokenizer
# unique_tokens = set(token.item() for seq in tokens['input_ids'] for token in seq)
# vocabulary_size = len(unique_tokens)

# print("Vocabulary Size:", vocabulary_size)
# # Assuming 'input_ids' is the first element in xtrain
# max_sequence_length_x = max(len(seq) for seq in xtrain)

# # Print or use the max_sequence_length_x in your pad_sequences and Embedding layer
# print("Max Sequence Length:", max_sequence_length_x)


In [4]:
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.models import Model
xtrain_np=xtrain.numpy()
input_length = None
input_layer = Input(shape=(input_length,))
embedding_layer = Embedding(input_dim=51000, output_dim=100, input_length=input_length)(input_layer)
conv1d_layer = Conv1D(128, 5, activation='relu')(embedding_layer)
maxpool_layer = GlobalMaxPooling1D()(conv1d_layer)
dense_layer = Dense(64, activation='relu')(maxpool_layer)
dropout_layer = Dropout(0.5)(dense_layer)
output_layer = Dense(1, activation='sigmoid')(dropout_layer)

model = Model(inputs=input_layer, outputs=output_layer)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Assuming xtrain_np and ytrain are NumPy arrays
model.fit(xtrain_np, ytrain, epochs=5, batch_size=256)




Epoch 1/5


90/90 [==============================] - 60s 654ms/step - loss: 0.4010 - accuracy: 0.8272
Epoch 2/5
90/90 [==============================] - 59s 650ms/step - loss: 0.0536 - accuracy: 0.9836
Epoch 3/5
90/90 [==============================] - 66s 736ms/step - loss: 0.0170 - accuracy: 0.9960
Epoch 4/5
90/90 [==============================] - 69s 763ms/step - loss: 0.0066 - accuracy: 0.9991
Epoch 5/5
90/90 [==============================] - 66s 729ms/step - loss: 0.0030 - accuracy: 0.9998


In [13]:
# Define your model
input_length = None
input_layer = Input(shape=(input_length,))
embedding_layer = Embedding(input_dim=51000, output_dim=100, input_length=input_length)(input_layer)
conv1d_layer = Conv1D(128, 5, activation='relu')(embedding_layer)
maxpool_layer = GlobalMaxPooling1D()(conv1d_layer)
dense_layer = Dense(64, activation='relu')(maxpool_layer)
dropout_layer = Dropout(0.5)(dense_layer)
output_layer = Dense(1, activation='sigmoid')(dropout_layer)

model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Perform cross-validation
# Perform cross-validation
accuracies = []
for train_index, test_index in kfold.split(xtrain_np):
    xtrain_cv, xval_cv = xtrain_np[train_index], xtrain_np[test_index]
    ytrain_cv, yval_cv = ytrain.iloc[train_index], ytrain.iloc[test_index]

    # Fit the model on the training data with validation data
    model.fit(xtrain_cv, ytrain_cv, epochs=5, batch_size=256, 
              validation_data=(xval_cv, yval_cv), callbacks=[early_stopping])

    # Evaluate the model on the testing data
    loss, accuracy = model.evaluate(xtest_cv, ytest_cv)
    accuracies.append(accuracy)

# Print the average accuracy
average_accuracy = sum(accuracies) / len(accuracies)
print("Average accuracy: ", average_accuracy)


Epoch 1/5
72/72 [==============================] - 62s 845ms/step - loss: 0.4778 - accuracy: 0.7849 - val_loss: 0.1319 - val_accuracy: 0.9619
Epoch 2/5
72/72 [==============================] - 60s 839ms/step - loss: 0.0747 - accuracy: 0.9772 - val_loss: 0.0448 - val_accuracy: 0.9866
Epoch 3/5
72/72 [==============================] - 61s 845ms/step - loss: 0.0221 - accuracy: 0.9948 - val_loss: 0.0402 - val_accuracy: 0.9873
Epoch 4/5
72/72 [==============================] - 61s 841ms/step - loss: 0.0079 - accuracy: 0.9990 - val_loss: 0.0377 - val_accuracy: 0.9888
Epoch 5/5
143/143 [==============================] - 3s 22ms/step - loss: 0.0385 - accuracy: 0.9884
Epoch 1/5
72/72 [==============================] - 60s 837ms/step - loss: 0.0134 - accuracy: 0.9964 - val_loss: 0.0026 - val_accuracy: 0.9998
Epoch 2/5
72/72 [==============================] - 62s 857ms/step - loss: 0.0039 - accuracy: 0.9995 - val_loss: 0.0014 - val_accuracy: 0.9998
Epoch 3/5
72/72 [==============================]

In [5]:
xtest_np=xtest.numpy()
model.evaluate(xtest_np,ytest)

179/179 [==============================] - 3s 18ms/step - loss: 0.0306 - accuracy: 0.9905


[0.030621709302067757, 0.9905396103858948]

In [14]:
model.save("dmodel12.keras")
model.save_weights('dmodel12_weights.keras')

In [7]:
x=tokens['input_ids']
y=df['generated']
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=42)
xtrain_np=xtrain.numpy()
xtest_np=xtest.numpy()
xtrain_np12=xtrain_np
xtest_np12=xtest_np
import numpy as np
xtrain_np12 = np.array(xtrain_np12)
xtest_np12=np.array(xtest_np12)
np.savetxt('B:/project/test/xtest_np12.csv', xtest_np12, delimiter=',')
np.savetxt('B:/project/train/xtrain_np12.csv', xtrain_np12, delimiter=',')
ytest.to_csv("B:/project/ytest/ytest12.csv")

In [8]:
x=tokens['input_ids']
y=df['generated']
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=42)
ytrain.to_csv("B:/project/ytrain/ytrain12.csv")
